In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
os.chdir('..')
import warnings
warnings.filterwarnings('ignore')

from tqdm.notebook import tqdm
import joblib
import numpy as np
import pandas as pd
import lightgbm as lgb

import matplotlib.pyplot as plt
%matplotlib inline

from m5.read import build_base_dataset
from m5.metric import WRMSSE
from m5.constants import ID_COLUMNS
from m5.funcs import only_days_columns
from utils.dtype import downcast

In [ ]:
import logging
logging.basicConfig(format="[%(asctime)s] %(levelname)s: %(message)s")
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

In [ ]:
ds = build_base_dataset()

In [ ]:
metric = WRMSSE(ds[ds['d'] < 1914], ds[ds['d'] >= 1914])

In [ ]:
# затираем реальные продажи и цену, дабы не ликнуть данные
ds['sold'] = ds['sold'].astype(np.float32)
ds.loc[:, 'target_sold'] = ds.loc[:, 'sold']
ds.loc[ds['d'] >= 1914, 'sold'] = np.nan

ds['sell_price'] = ds['sell_price'].astype(np.float32)

# Подготовка набора данных

## Исключение данных "до старта продаж"

In [ ]:
# определяем стартпродаж по критерию - как только был продана хотяб 1 единца - старт т наступил
ds['start_of_sales'] = ds.groupby('id')['sold'].transform(lambda x: np.argmax(x > 0))
# поскольку сдвиг агрегатов у нас минимум 28 дней, то сдвинем старт продаж на этот срок
ds = ds[ds['d'] > (ds['start_of_sales'])]

In [ ]:
def calculate_features(ds: pd.DataFrame, delta: int):
    group_by_id = ds.sort_values(by='d').groupby('id')
    features = pd.DataFrame({
        f'sold_previous_{delta}': group_by_id['sold'].transform(lambda x: x.shift(1 + delta)).values,
        f'sold_mean_3_{delta}': group_by_id['sold'].transform(lambda x: x.shift(1 + delta).rolling(3).mean()).values,
        f'sold_mean_7_{delta}': group_by_id['sold'].transform(lambda x: x.shift(1 + delta).rolling(7).mean()).values,
        f'sold_mean_14_{delta}': group_by_id['sold'].transform(lambda x: x.shift(1 + delta).rolling(14).mean()).values,
        f'sold_mean_28_{delta}': group_by_id['sold'].transform(lambda x: x.shift(1 + delta).rolling(28).mean()).values,
        f'sold_mean_56_{delta}': group_by_id['sold'].transform(lambda x: x.shift(1 + delta).rolling(56).mean()).values,
        f'sold_mean_182_{delta}': group_by_id['sold'].transform(lambda x: x.shift(1 + delta).rolling(182).mean()).values,
        f'price_previous_{delta}': group_by_id['sell_price'].transform(lambda x: x.shift(1 + delta)).values,
        f'price_diff_3_{delta}': group_by_id['sell_price'].transform(lambda x: x.diff(3).shift(delta)).values,
        f'price_diff_7_{delta}': group_by_id['sell_price'].transform(lambda x: x.diff(7).shift(delta)).values,
        f'price_diff_28_{delta}': group_by_id['sell_price'].transform(lambda x: x.diff(28).shift(delta)).values
    })
    return features

In [ ]:
%%time
features_0 = calculate_features(ds, delta=7 * 0)
features_1 = calculate_features(ds, delta=7 * 1)
features_2 = calculate_features(ds, delta=7 * 2)
new_features = features_0.columns.tolist() + features_1.columns.tolist() + features_2.columns.tolist()

In [ ]:
ds = ds.reset_index(drop=True)
ds = pd.concat([ds, features_0, features_1, features_2], axis=1)

In [ ]:
targets_shift_col = [f'target_{lag}' for lag in range(0, 28)]
target_lag_path = 'data/cache/target_lag.jbl'

if os.path.isfile(target_lag_path):
    logging.info('Use cache.')
    targets_shift = joblib.load(target_lag_path).reset_index(drop=True)[targets_shift_col]
    ds = pd.concat([ds, targets_shift], axis=1)
else:
    logging.info('Re-build sold lag features.')
    group_by_id = ds.sort_values(by='d')[['id', 'target_sold']].groupby('id')
    for lag in tqdm(range(0, 28)):
        ds[f'target_{lag}'] = group_by_id['target_sold'].transform(lambda x: x.shift(-lag))
        
    logging.info(f'Save to: {target_lag_path}')
    joblib.dump(ds[ID_COLUMNS + targets_shift_col], target_lag_path)

In [ ]:
ds = ds.dropna(subset=new_features + targets_shift_col)
ds = downcast(ds)

# Обучение модели

In [ ]:
ds['event_type_1'] = ds['event_type_1'].cat.add_categories('NaN').fillna('NaN')
ds['event_name_1'] = ds['event_name_1'].cat.add_categories('NaN').fillna('NaN')
ds['event_type_2'] = ds['event_type_2'].cat.add_categories('NaN').fillna('NaN')
ds['event_name_2'] = ds['event_name_2'].cat.add_categories('NaN').fillna('NaN')

In [ ]:
CATEGORICAL_FEATURES = [
    'item_id', 'dept_id','store_id', 'cat_id', 'state_id',
    'weekday',
    'event_name_1', 'event_name_2', 'event_type_1', 'event_type_2',
    'snap_CA', 'snap_TX', 'snap_WI',
]

NUMERICAL_FEATURES = [
    'sell_price'
]+ new_features

XS = CATEGORICAL_FEATURES + NUMERICAL_FEATURES
TARGET = 'target_sold'

In [ ]:
train_ds = ds[(ds['d'].between(818, 1913))]
val_ds = ds[ds['d'] >= 1914]
train = train_ds[XS + targets_shift_col].dropna()
val = val_ds[XS + targets_shift_col].dropna()

In [ ]:
train.shape, val.shape

In [ ]:
params = {
    'objective': 'poisson',
    'num_iterations': 2000,
    'learning_rate': 0.075,
    'verbose': 20,
    'bagging_fraction': 0.5,
    'feature_fraction': 0.7,
    'metric': ['rmse']
}

In [ ]:
models_day = {}
for day_shift in range(0, 28):
    logging.info(f'Train model for {day_shift} day.')
    target = f'target_{day_shift}'
    cat_train_set = lgb.Dataset(train[XS], train[target])
    cat_valid_set = lgb.Dataset(val[XS], val[target])    
    

    logging.info('Starting model train')
    evals_result = {}
    model = lgb.train(
        params,
        train_set=cat_train_set,
        valid_sets=[cat_train_set, cat_valid_set],
        early_stopping_rounds=10,
        categorical_feature=CATEGORICAL_FEATURES,
        evals_result=evals_result,
        verbose_eval=20 
    )
    models_day[day_shift] = {
        'model': model,
        'evals_result': evals_result
    }
    logging.info('The model is trained')

In [ ]:
fig, axs = plt.subplots(4, 7, figsize=(25, 15))
for idx in range(0, 28):
    row = idx // 7
    col = idx % 7
    ax = axs[row, col]
    
    model_evals = models_day[0]['evals_result']
    lgb.plot_metric(model_evals, metric='rmse', ax=ax)
    ax.set_title(f'Day: {idx + 1}')
fig.tight_layout()

In [ ]:
val_pred = val_ds[['constant_id', 'id'] + XS].dropna()

for idx_, model_dict in models_day.items():
    model = model_dict['model']
    predict = model.predict(val_pred[XS])
    val_pred.loc[:, f'd_{1914 + idx_}'] = predict

val_pred = val_pred[ID_COLUMNS + val_pred.columns[val_pred.columns.str.startswith('d_')].tolist()]
print('WRMSSE: ', metric.score(val_pred))

# Отправка сабмита

In [ ]:
submit_days = only_days_columns(val_pred)
submit_evaluation = val_pred[['id'] + submit_days]
submit_rename_dict = {
    col: f'F{i}' 
    for i, col in enumerate(only_days_columns(submit_evaluation), start=1)
}
submit_evaluation = submit_evaluation.rename(columns=submit_rename_dict)

submit_validation = submit_evaluation.copy()
submit_validation['id'] = submit_validation.id.str.replace('evaluation', 'validation')

submit = pd.concat([submit_evaluation, submit_validation])

In [ ]:
submit.to_csv('submit.csv', index=False)

In [ ]:
!kaggle competitions submit -c m5-forecasting-accuracy -f submit.csv -m "Daily predict"